# Bibtex to DF and tidy

**One off code for data rationalization and normalization.**

* df
* author list
* publisher list
* journal list
* files
* normalization

item 241 has weird annot


## Provenance
* Created new 2025-05-16 at Bignor!

In [6]:
from pathlib import Path 
import pandas as pd
import re
from functools import partial
import pprint as pp

import numpy as np
import Levenshtein
import sys
sys.path.append('..')
import archivum.mendeley_port as sc
# from scratch import safeyear
import great2_bibtex as scbt

from greater_tables import GT
fGT = partial(GT, large_ok=True, formatters={'index': str, 'year': str})
%load_ext autotime

C:\Users\steve\AppData\Local\archivum
time: 0 ns (started: 2025-05-23 21:05:53 +01:00)


## Manual read and manual inspection

In [7]:
p = Path('books.bib')
p = Path('library.bib')
p.exists()

True

time: 0 ns (started: 2025-05-23 21:05:55 +01:00)


In [8]:
b = sc.Bib2df(p, False)
len(b.df)

uber regex sub found n = 517 replacements


6118

time: 563 ms (started: 2025-05-23 21:05:56 +01:00)


In [9]:
b.df.head(2).T

,1,2
type,article,book
tag,Schmidt2003,Bahnemann2015
abstract,It is shown that the arbitrage free portfolio paradigm being applied to a portfolio with an arbi...,NaN
author,"Schmidt, Anatoly B.","Bahnemann, David"
doi,10.1016/S0378-4371(02)01551-0,NaN
file,:C\:/S/Library/Schmidt/2003_The true invariant of an arbitrage free portfolio.pdf:pdf,:C\:/S/Library/Bahnemann/2015_Distributions for Actuaries.pdf:pdf;:C\:/S/Library/Bahnemann/2015_...
issn,03784371,NaN
journal,Physica A: Statistical Mechanics and its Applications,NaN
keywords,"Arbitrage free portfolio,Option pricing",NaN
pages,535--538,NaN


time: 0 ns (started: 2025-05-23 21:05:57 +01:00)


In [10]:
# filled values for each column  
b.contents()

,column,nonna,distinct
0,type,6118,10
1,tag,6118,6076
2,abstract,2518,2375
3,author,6113,4856
4,doi,2687,2538
5,file,5951,5917
6,issn,2425,826
7,journal,3475,1224
8,keywords,2999,2052
9,pages,3885,2748


time: 62 ms (started: 2025-05-23 21:06:05 +01:00)


In [ ]:
# value counts for each field to look at the data
for f in b.contents().column: 
    display(fGT(b.df[f].value_counts().sort_values()[::-1].head(30), caption=f))
    print()

In [ ]:
# distinct col c
j = b.distinct('journal')
j[:10]

# Author Names

1. remove inital punctuation
1. de-tex (unicode)
1. find longest unique extension
1. add back accents (slight hack)
1. --> proposed


In [53]:
b._author_map_df = None 
adf = b.author_map_df

Field: authors
Decode errors: len(self.last_decode) = 0
time: 1.17 s (started: 2025-05-23 21:41:00 +01:00)


In [55]:
adf.iloc[20:25]

,original,unicoded,longest,accents,proposed
20,"Abbott, B P","Abbott, B P","Abbott, B P","Abbott, B P","Abbott, B. P."
21,"Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed","Abdel-Hameed, Mohamed"
22,"Abdikerimova, Samal","Abdikerimova, Samal","Abdikerimova, Samal","Abdikerimova, Samal","Abdikerimova, Samal"
23,"Abowd, John M.","Abowd, John M","Abowd, John M","Abowd, John M","Abowd, John M."
24,"Abraham, Andreas","Abraham, Andreas","Abraham, Andreas","Abraham, Andreas","Abraham, Andreas"


time: 15 ms (started: 2025-05-23 21:41:05 +01:00)


In [57]:
len(adf.query('longest != unicoded'))

409

time: 0 ns (started: 2025-05-23 21:41:34 +01:00)


In [58]:
adf.query('longest != unicoded').sample(20)

,original,unicoded,longest,accents,proposed
2383,"Ghossoub, M","Ghossoub, M","Ghossoub, Mario","Ghossoub, Mario","Ghossoub, Mario"
3828,"Lai, Gene","Lai, Gene","Lai, Gene C","Lai, Gene C","Lai, Gene C."
1518,"Dan{\'{i}}elsson, J","Daníelsson, J","Daníelsson, Jón","Daníelsson, Jón","Daníelsson, Jón"
1123,"Chen, John","Chen, John","Chen, John Donghui","Chen, John Donghui","Chen, John Donghui"
1656,"Diamond, Douglas W.","Diamond, Douglas W","Diamond, Douglas W","Diamond, Douglas W","Diamond, Douglas W."
13,"Aas, K","Aas, K","Aas, Kjersti","Aas, Kjersti","Aas, Kjersti"
1112,"Chatterjee, S.","Chatterjee, S","Chatterjee, Sourav","Chatterjee, Sourav","Chatterjee, Sourav"
8132,"{Van Loan}, Charles","{Van Loan}, Charles","{Van Loan}, Charles F","{Van Loan}, Charles F","{Van Loan}, Charles F."
3239,"Jiao, Z","Jiao, Z","Jiao, Zhanyi","Jiao, Zhanyi","Jiao, Zhanyi"
6550,"Spielman, Daniel","Spielman, Daniel","Spielman, Daniel A","Spielman, Daniel A","Spielman, Daniel A."


time: 15 ms (started: 2025-05-23 21:42:10 +01:00)


In [60]:
adf.query('longest != accents').sample(20)

,original,unicoded,longest,accents,proposed
103,"Albrecher, Hansjorg","Albrecher, Hansjorg","Albrecher, Hansjorg","Albrecher, Hansjörg","Albrecher, Hansjörg"
977,"Candes, Emmanuel J.","Candes, Emmanuel J","Candes, Emmanuel J","Candès, Emmanuel J","Candès, Emmanuel J."
2634,"Guillen, Montserrat","Guillen, Montserrat","Guillen, Montserrat","Guillén, Montserrat","Guillén, Montserrat"
1513,"Danielsson, Jon","Danielsson, Jon","Danielsson, Jon","Daníelsson, Jón","Daníelsson, Jón"
3986,"Letac, Gerard","Letac, Gerard","Letac, Gerard","Letac, Gérard","Letac, Gérard"
86,"Ait-Sahalia, Yacine","Ait-Sahalia, Yacine","Ait-Sahalia, Yacine","Aït-Sahalia, Yacine","Aït-Sahalia, Yacine"
3692,"Kratschmer, Volker","Kratschmer, Volker","Kratschmer, Volker","Krätschmer, Volker","Krätschmer, Volker"
5959,"Ruszczynski, Andrzej","Ruszczynski, Andrzej","Ruszczynski, Andrzej","Ruszczyński, Andrzej","Ruszczyński, Andrzej"
1116,"Chavez-Demoulin, Valerie","Chavez-Demoulin, Valerie","Chavez-Demoulin, Valerie","Chavez-Demoulin, Valérie","Chavez-Demoulin, Valérie"
2005,"Farach-Colton, Martin","Farach-Colton, Martin","Farach-Colton, Martin","Farach-Colton, Martín","Farach-Colton, Martín"


time: 16 ms (started: 2025-05-23 21:42:44 +01:00)


# edition

In [76]:
for v in sorted(b.distinct('edition')):
    print(f'"{v}": "{v.title()}",')

"10": "10",
"2": "2",
"2nd": "2Nd",
"2nd Editio": "2Nd Editio",
"3": "3",
"3rd": "3Rd",
"5": "5",
"Annals of": "Annals Of",
"Enlarged": "Enlarged",
"Fifth": "Fifth",
"First": "First",
"Fourth": "Fourth",
"Lecture No": "Lecture No",
"Ninth": "Ninth",
"Second": "Second",
"Second Edi": "Second Edi",
"Seventh": "Seventh",
"Sixth": "Sixth",
"Third": "Third",
"fourth": "Fourth",
time: 15 ms (started: 2025-05-23 22:04:39 +01:00)


# mendeley-tags
Just carry over

# Porting Menedeley

In [11]:
b = sc.Bib2df(p, True)

uber regex sub found n = 517 replacements
time: 578 ms (started: 2025-05-23 21:06:09 +01:00)


In [45]:
b.port_mendeley_file()

Field: authors
Decode errors: len(self.last_decode) = 0
fixed authors
Field: title
len(self.last_decode) = 37
Fixed title
Field: journal
len(self.last_decode) = 4
Fixed journal
Fixed publisher
Fixed institution
Fixed booktitle
Fixed address
Fixed editor
time: 14.9 s (started: 2025-05-21 11:41:38 +01:00)


In [ ]:
b.show_unicode_errors()

In [ ]:
b.contents(ported=True)

In [ ]:
# run fillna = False
df = b.contents(True)

In [56]:
# check some mappings
fGT(b.ported_df.loc[b.ported_df.author.str.contains('Can'), ['author', 'title']])

index,author,title
181,"Marucho, M. and Canal, C. A.Garcia and Fanchiotti, Huner",{The Landau distribution for charged particles traversing thin films}
599,"Canis, Bill",{Motor Vehicle Supply Chain: Effects of the Japanese Earthquake and Tsunami (Google eBook)}
937,"Bellare, M. and Canetti, R. and Krawczyk, H.",{Keying hash functions for message authentication}
944,"Candès, Emmanuel J.",{Mathematics of sparsity (and a few other things)}
1159,"Candès, Emmanuel J.",{Compressive sampling}
1675,"Gangbo, Wilfrid and McCann, Robert J.",{The geometry of optimal transportation}
1727,"Cannas, Giuseppina",{A stochastic model for the sustainable investment policy in a defined benefit pension fund}
1783,Canon,{PowerShot G15 Manual}
1848,"Candès, Emmanuel J. and Wakin, M. B.",{An Introduction To Compressive Sampling}
2241,Canon,{Canon 5DSR Instruction Manual}


time: 110 ms (started: 2025-05-21 11:56:27 +01:00)


In [54]:
b.no_file()

,tag,type,author,title,year,journal,file
38,keynesgeneral,book,"Keynes, John Maynard","{General theory of employment, interest and mo...",1936,,
53,Mildenhall2018c,article,"Mildenhall, Stephen J",{Every law invariant coherent risk measure has...,2018,,
75,Eling2017,article,"Eling, Martin and Jia, Ruo",{Recent Research Developments Affecting Nonlif...,2017,Risk Management and Insurance Review,
114,daykin94,book,"Daykin, C. D. and Pentikainen, T. and Pesonen, M.",{Practical Risk Theory for Actuaries},1994,,
133,Crouhy2010,article,"Crouhy, Michel and Galai, Dan and Mark, Robert",{Risk Exposures},2010,Encyclopedia of Quantitative Finance,
...,...,...,...,...,...,...,...
5865,hull01,book,"Hiriart-Urruty, Jean-Baptiste and Lemar{\'{e}}...",{Fundamentals of Convex Analysis},2001,,
5931,eeckhoudtgs04,book,"Eeckhoudt, Louis and Gollier, Christian and Sc...",{Economic and Financial Decisions under Risk},2005,,
6006,Strain1997,book,"Strain, Robert W.",{Reinsurance},1997,,
6039,jacob02,book,"Jacob, Niels",{Pseduo Differential Operators & Markov Proces...,2002,,


time: 32 ms (started: 2025-05-21 11:50:59 +01:00)


# Duplicated tags

In [83]:
df = b.ported_df.copy().sort_values(['author', 'year'])

time: 31 ms (started: 2025-05-21 12:20:45 +01:00)


In [72]:
b.df.loc[[2841]].T

,2841
type,book
tag,Jouini2001a
abstract,
author,
doi,
file,":C\:/S/Library/Unknown/2001_Option Pricing, In..."
issn,
journal,
keywords,Book
pages,


time: 15 ms (started: 2025-05-21 12:15:15 +01:00)


In [179]:
# suggested tag
df['stag'] = sc.suggest_tag(df) 

time: 406 ms (started: 2025-05-21 17:32:47 +01:00)


In [180]:
df['tag_base'] = df.tag.str.replace(
    r'^(.*?[0-9]{4})[a-z]+$', r'\1', regex=True)

time: 47 ms (started: 2025-05-21 17:32:48 +01:00)


In [ ]:
fGT(df.query('tag_base != stag').sort_values('tag_base')[['tag', 'tag_base', 'stag', 'year']])

In [181]:
df['tag_proposed'] = df['stag']
# create here
ka = sc.KeyAllocator(df.tag)
df.loc[df.tag_base != df.stag, 'tag_proposed'] = df.loc[df.tag_base != df.stag, 'tag_proposed'].map(ka)

time: 31 ms (started: 2025-05-21 17:32:49 +01:00)


In [182]:
df.query('tag_base != stag')[['tag', 'tag_base', 'stag', 'tag_proposed', 'author', 'title', 'year']].head(50)

,tag,tag_base,stag,tag_proposed,author,title,year
5196,Deregulation2002,Deregulation2002,Cummins2002,Cummins2002b,,{Deregulating property - Liability Insurance: ...,2002
1005,AB_IRS_12,AB_IRS_12,Mildenhall2012,Mildenhall2012a,,{Insurance Risk Study},2012
3745,AIP2015,AIP2015,AIPSO2015,AIPSO2015,AIPSO,{AIPSO Facts 2014/15},2015
4020,Manuals,Manuals,AIPSO2016,AIPSO2016,AIPSO,{AIPSO Facts 2015/16},2016
5517,Statistics1992,Statistics1992,Aalen1992,Aalen1992,"Aalen, Odd O.",{Modelling Heterogeneity in Survival Analysis ...,1992
4721,Abdel-Hameed2014,Abdel-Hameed2014,AbdelHameed2014,AbdelHameed2014,"Abdel-Hameed, Mohamed",{Lévy Processes and Their Applications in Reli...,2014
953,Abdikerimova2024,Abdikerimova2024,Abdikerimova,Abdikerimova,"Abdikerimova, Samal and Boonen, Tim J. and Fen...",{Multiperiod peer-to-peer risk sharing},
461,abrahammr88,abrahammr88,Abraham1988,Abraham1988,"Abraham, R. and Marsden, J. E. and Ratiu, T.","{Manifolds, Tensor Analysis, and Applications}",1988
392,as,as,Abramowitz1970,Abramowitz1970,"Abramowitz, Milton and Stegun, Irene A.",{Handbook of Mathematical Functions},1970
1949,Abreu,Abreu,Abreu1988,Abreu1988a,"Abreu, Dilip",{On the theory of infinitely repeated games wi...,1988


time: 47 ms (started: 2025-05-21 17:32:50 +01:00)


In [1]:
10

10

In [4]:
import socket
socket.gethostname()

'Cantor'

In [104]:
remove_accents('Aábeê')

'Aabee'

time: 15 ms (started: 2025-05-21 12:29:51 +01:00)


In [57]:
base_cols = ['tag', 'type', 'author', 'title', 'year', 'journal', 'file', 'publisher']
len(base_cols)

8

time: 16 ms (started: 2025-05-21 12:09:44 +01:00)


In [58]:
for t in sorted(b.df.loc[b.df.tag.duplicated(keep=False), base_cols].tag.value_counts().keys()):
    bit = b.df.drop(columns=['abstract', 'annote']).query('tag == @t').T.dropna(how='all')
    bit_eq = (bit.eq(bit.iloc[:, 0], axis=0).all(axis=1)).astype(int)
    bit['union'] = bit.apply(lambda x: x.iloc[0] if pd.isna(x.iloc[1]) else x.iloc[1], axis=1)
    bit['eq'] = bit_eq
    display(fGT(bit.loc[~bit.union.isna()].fillna(''), caption=t))

index,742,2288,union,eq
type,book,book,book,1
tag,Aliprantis2006,Aliprantis2006,Aliprantis2006,1
author,"Aliprantis, Charalambos D and Burkinshaw, Owen","Aliprantis, Charalambos D and Border, Kim","Aliprantis, Charalambos D and Border, Kim",0
doi,10.1007/978-94-015-9349-6_3,10.1007/3-540-29587-9,10.1007/3-540-29587-9,0
file,":C\:/S/Library/Aliprantis, Burkinshaw/2006_Positive Operators.pdf:pdf",":C\:/S/Library/Aliprantis, Border/2006_Infinite Dimensional Analysis A Hitchhiker's Guide.pdf:pdf",":C\:/S/Library/Aliprantis, Border/2006_Infinite Dimensional Analysis A Hitchhiker's Guide.pdf:pdf",0
issn,,,,1
journal,,,,1
keywords,,Book,Book,0
pages,,704,704,0
title,{Positive Operators},{Infinite Dimensional Analysis: A Hitchhiker's Guide},{Infinite Dimensional Analysis: A Hitchhiker's Guide},0


index,418,5541,union,eq
type,article,techreport,techreport,0
tag,AonBenfield2016,AonBenfield2016,AonBenfield2016,1
author,"Mildenhall, Stephen J",{Aon Benfield},{Aon Benfield},0
doi,,,,1
file,:C\:/S/Library/Mildenhall/2016_Insurance Risk Study The First Ten Years.pdf:pdf,:C\:/S/Library/Aon Benfield/2016_Cyber Opportunity and Future Growth.pdf:pdf,:C\:/S/Library/Aon Benfield/2016_Cyber Opportunity and Future Growth.pdf:pdf,0
issn,,,,1
journal,,,,1
keywords,"Aon,Mildenhall",Aon,Aon,0
pages,,,,1
title,{Insurance Risk Study: The First Ten Years},{Cyber Opportunity and Future Growth},{Cyber Opportunity and Future Growth},0


index,1793,3225,union,eq
type,article,unpublished,unpublished,0
tag,Avanzi2011,Avanzi2011,Avanzi2011,1
author,"Avanzi, Benjamin","Avanzi, Benjamin and Cassar, Luke C and Wong, Bernard","Avanzi, Benjamin and Cassar, Luke C and Wong, Bernard",0
doi,10.2143/AST.41.2.2136990,,,0
file,:C\:/S/Library/Avanzi/2011_Optimal dividends and capital injections in the dual model with diffusions.pdf:pdf,":C\:/S/Library/Avanzi, Cassar, Wong/2011_Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy C.pdf:pdf",":C\:/S/Library/Avanzi, Cassar, Wong/2011_Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy C.pdf:pdf",0
issn,05150361 17831350,,,0
journal,ASTIN Bulletin,,,0
keywords,,"c46,c51,c52,compound poisson process,dependence,g22,insurance,jel codes,l{\'{e}}vy copula,real data","c46,c51,c52,compound poisson process,dependence,g22,insurance,jel codes,l{\'{e}}vy copula,real data",0
pages,611--644,,,0
title,{Optimal dividends and capital injections in the dual model with diffusions},{Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas},{Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas Modelling Dependence in Insurance Claims Processes with L{\'{e}}vy Copulas},0


index,2023,2938,union,eq
type,article,article,article,1
tag,Babbel1997,Babbel1997,Babbel1997,1
author,"Babbel, David F.","Babbel, David F","Babbel, David F",0
doi,,,,1
file,,,,1
issn,,,,1
journal,North American Actuarial Journal,North American Actuarial Journal,North American Actuarial Journal,1
keywords,,,,1
pages,122--125,122--123,122--123,0
title,"{Discussion of ``Two Paradigms for The Market Value of Liabilities, Robert R. Reitano''}",{Review of Two Paradigms for the Market Value of Liabilities by Reitano},{Review of Two Paradigms for the Market Value of Liabilities by Reitano},0


index,3857,4082,union,eq
type,article,article,article,1
tag,Bailey1967,Bailey1967,Bailey1967,1
author,"Bailey, Robert A","Bailey, Robert A","Bailey, Robert A",1
doi,,,,1
file,:C\:/S/Library/Bailey/1967_Underwriting Profit From Investments.pdf:pdf,,,0
issn,,,,1
journal,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,Proceedings of the Casualty Actuarial Society,1
keywords,UNDERWRITING PROFIT FROM INVESTMENTS,,,0
pages,1--8,1--8,1--8,1
title,{Underwriting Profit From Investments},{Underwriting Profits From Investments},{Underwriting Profits From Investments},0


...

index,1732,2699,union,eq
type,article,article,article,1
tag,Vinogradov2004,Vinogradov2004,Vinogradov2004,1
author,"Vinogradov, Vladimir","Vinogradov, Vladimir","Vinogradov, Vladimir",1
doi,10.31390/cosa.2.2.02,10.1081/STA-120029821,10.1081/STA-120029821,0
file,:C\:/S/Library/Vinogradov/2004_Properties of certain L{\'{e}}vy and geometric L{\'{e}}vy processes.pdf:pdf;:C\:/S/Library/Vinogradov/2004_Properties of certain L{\'{e}}vy and geometric L{\'{e}}vy processes.pdf:pdf,:C\:/S/Library/Vinogradov/2004_On the power-variance family of probability distributions.pdf:pdf,:C\:/S/Library/Vinogradov/2004_On the power-variance family of probability distributions.pdf:pdf,0
issn,0973-9599,"3,610,926","3,610,926",0
journal,Communications on Stochastic Analysis,Communications in Statistics - Theory and Methods,Communications in Statistics - Theory and Methods,0
keywords,,"Cumulant,Decomposition criterion,Indecomposable component,Power-variance family,Tweedie law,Weak convergence","Cumulant,Decomposition criterion,Indecomposable component,Power-variance family,Tweedie law,Weak convergence",0
pages,,1007--1029,1007--1029,0
title,{Properties of certain L{\'{e}}vy and geometric L{\'{e}}vy processes},{On the power-variance family of probability distributions},{On the power-variance family of probability distributions},0


index,1185,1961,union,eq
type,article,article,article,1
tag,Wang2000,Wang2000,Wang2000,1
author,"Wang, Shaun S.","Wang, Shaun S.","Wang, Shaun S.",1
doi,,10.2307/253675,10.2307/253675,0
file,:C\:/S/Library/Wang/2000_A class of distorion operators for pricing financial and insurance risks.pdf:pdf,:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks.pdf:pdf;:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks(2).pdf:pdf,:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks.pdf:pdf;:C\:/S/Library/Wang/2000_A Class of Distortion Operators for Pricing Financial and Insurance Risks(2).pdf:pdf,0
issn,,"224,367","224,367",0
journal,Journal of Risk and Insurance,The Journal of Risk and Insurance,The Journal of Risk and Insurance,0
keywords,WangSS,WangSS,WangSS,1
pages,15--36,15--36,15--36,1
title,{A class of distorion operators for pricing financial and insurance risks},{A Class of Distortion Operators for Pricing Financial and Insurance Risks},{A Class of Distortion Operators for Pricing Financial and Insurance Risks},0


index,1225,1356,union,eq
type,article,article,article,1
tag,Wang2018,Wang2018,Wang2018,1
author,"Wang, Ruodu and Wei, Yunran and Willmot, Gordon","Wang, Ruodu and Wei, Yunran and Willmot, Gordon","Wang, Ruodu and Wei, Yunran and Willmot, Gordon",1
doi,10.2139/ssrn.2956962,10.2139/ssrn.2956962,10.2139/ssrn.2956962,1
file,":C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals(2).pdf:pdf;:C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals.pdf:pdf",":C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals(2).pdf:pdf",":C\:/S/Library/Wang, Wei, Willmot/2018_Characterization, Robustness and Aggregation of Signed Choquet Integrals(2).pdf:pdf",0
issn,,,,1
journal,SSRN Electronic Journal,SSRN Electronic Journal,SSRN Electronic Journal,1
keywords,"PMIR,WangR,choquet integrals,comonotonicity,risk aggregation,risk functionals,robustness","WangR,choquet integrals,comonotonicity,risk aggregation,risk functionals,robustness","WangR,choquet integrals,comonotonicity,risk aggregation,risk functionals,robustness",0
pages,,,,1
title,"{Characterization, Robustness and Aggregation of Signed Choquet Integrals}","{Characterization, Robustness and Aggregation of Signed Choquet Integrals}","{Characterization, Robustness and Aggregation of Signed Choquet Integrals}",1


index,2488,5380,union,eq
type,article,article,article,1
tag,Yin2018,Yin2018,Yin2018,1
author,"Yin, Chuancun and Zhu, Dan","Yin, Chuancun and Zhu, Dan","Yin, Chuancun and Zhu, Dan",1
doi,,10.4236/jfrm.2018.71002,10.4236/jfrm.2018.71002,0
file,,":C\:/S/Library/Yin, Zhu/2018_New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR.pdf:pdf",":C\:/S/Library/Yin, Zhu/2018_New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR.pdf:pdf",0
issn,,2167-9533,2167-9533,0
journal,Journal of Financial Risk Management,Journal of Financial Risk Management,Journal of Financial Risk Management,1
keywords,,"additivity,coherent risk measure,comonotonicity,copula,distortion functions,distortion risk measure,extreme value theory,gluevar,maximum domain of attraction,spectral risk measure,super,tail distortion risk measure,tail sub,tvar","additivity,coherent risk measure,comonotonicity,copula,distortion functions,distortion risk measure,extreme value theory,gluevar,maximum domain of attraction,spectral risk measure,super,tail distortion risk measure,tail sub,tvar",0
pages,12--38,12--38,12--38,1
title,{No Title},{New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR},{New Class of Distortion Risk Measures and Their Tail Asymptotics with Emphasis on VaR},0


time: 6.53 s (started: 2025-05-21 12:09:44 +01:00)
